In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [2]:
import pandas as pd
from pandas import DataFrame

import numpy as np


data = pd.read_csv('data/showingTelling_csv.csv', delimiter = ',', encoding="ISO-8859-1")

data.index.name = "index"
data.columns = ["label", "text"]
data

,label,text
index,,
0,0,John was sad to see his girlfriend leave.
1,0,The house was creepy.
2,0,I heard footsteps creeping behind me and it ma...
3,0,She was my best friend. I could tell her almos...
4,0,She hated it there because it smelled bad.
...,...,...
254,1,"Her hand reached for the massive, iron door ha..."
255,1,The way the door decisively slammed behind her...
256,1,Dust coated every last surface. He ran his fin...


In [4]:
from sklearn.model_selection import train_test_split

def shuffle(df, n=1, axis=0): #데이터가 1111 0000이라 잘 섞어주자. 
    df = data.copy()
    for _ in range(n):
        df.apply(np.random.shuffle, axis=axis)
    return df

shuffle(data)

train_df, test_df = train_test_split(data, test_size=0.2)
print(len(train_df))
print(len(test_df))

207
52


In [5]:
import re
import pandas
import numpy
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [6]:
def preprocessing(review, remove_stopwords=False):
        
    # 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    review_text = re.sub("[^a-zA-Z]", " ", review)

    # 대문자들을 소문자로 바꾸고 공백단위로 텍스트들 나눠서 리스트로 만든다.
    words = review_text.lower().split()

    if remove_stopwords: 
        # 불용어들을 제거
    
        #영어에 관련된 불용어 불러오기
        stops = set(stopwords.words("english"))
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        # 단어 리스트를 공백을 넣어서 하나의 글로 합친다.
        clean_review = ' '.join(words)

    else: # 불용어 제거하지 않을 때
        clean_review = ' '.join(words)

    return clean_review

In [8]:
clean_train_ = []
for review in train_df['text']:
    clean_train_.append(preprocessing(review, remove_stopwords=True))

# 전처리된 데이터 확인. 잘됨 !! ㅎ
clean_train_[0]

'katie picks bottom comes away'

In [9]:
clean_train_[:5]

['katie picks bottom comes away',
 'lime green patio umbrella flapped happily breeze covered strawberry slushies watermelon pies bright cheerful stacks donuts emily stepped outside feature crystal pitcher pink lemonade spray warm cookies center table favorite summer dress knew soon stepped patio gate life would never',
 'first spring stumbled darkness war florentine coffee house served waitress truly like sun ophelia diagilo dressed yellow spreading warmth promise sex passed frothy cappuccino',
 'boots crunched walked behind bar gholy shit h screamed saw pool blood',
 'heard footsteps creeping behind made whole situation scarier']

In [11]:
clean_test_ = []
for review in test_df['text']:
    clean_test_.append(preprocessing(review, remove_stopwords=True))
    
# 전처리된 데이터 확인. 잘됨 !! ㅎ
clean_test_[0]

'heart beating fast made ethump thump thump f'

In [12]:
clean_test_[:5]

['heart beating fast made ethump thump thump f',
 'another metres mary looked sheer cliff face remaining metres head arms ached kept climbing moving slowly steadily one foothold next could give exhaustion reached safety',
 'stood mesmerized white caps ocean waves dug sand grasping one breath life',
 'single dim candle lit room house smelled like dust rotting wood something faintly metallic made john think blood stuffed animals mounted around room wild eyed buck grizzly frozen fury screech owl sharp yellow talons',
 'james tall attractive women charming fell immediately never guessed little cared']

In [14]:
print(train_df["label"]) #showing telling 컬럼값을 확인해보고

index
179    1
257    1
247    1
183    1
2      0
      ..
250    1
230    1
242    1
61     0
155    1
Name: label, Length: 207, dtype: int64


In [16]:
train_df['cleaned_text'] = clean_train_ # 이제 전처리된 내용을 한눈에 비교해 볼 수 있다.
train_df[:5] #데이터 앞부분 5개반 확인해보자

/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,label,text,cleaned_text
index,,,
179,1,Katie picks at the bottom and it comes away.,katie picks bottom comes away
257,1,The lime green patio umbrella flapped happily ...,lime green patio umbrella flapped happily bree...
247,1,"The first spring of 1946, he had stumbled out ...",first spring stumbled darkness war florentine ...
183,1,His boots crunched as he walked behind the bar...,boots crunched walked behind bar gholy shit h ...
2,0,I heard footsteps creeping behind me and it ma...,heard footsteps creeping behind made whole sit...


In [18]:
train_dataset = train_df[['label', 'cleaned_text']] #전처리 1차 끝!
train_dataset[:5]

,label,cleaned_text
index,,
179,1,katie picks bottom comes away
257,1,lime green patio umbrella flapped happily bree...
247,1,first spring stumbled darkness war florentine ...
183,1,boots crunched walked behind bar gholy shit h ...
2,0,heard footsteps creeping behind made whole sit...


In [24]:
train_dataset.dropna(inplace=True)
#test_dataset.dropna(inplace=True)

train_df = train_dataset.sample(frac=1, random_state=999)
#test_df = test_dataset.sample(frac=1, random_state=999)

In [25]:
train_df

,label,cleaned_text
index,,
149,1,butterflies stomach
185,1,yellow eyes glowing red long black fur stood e...
238,1,gnot wearin f ring see h driver said boarded
176,1,glances looks sharply
100,0,spring came back war florentine coffee house s...
...,...,...
161,1,papa going ax said fern mother
205,1,jay waited whole day go reading class always s...
145,1,bright red ribbons dimly lit candles green clo...


In [26]:
class STDataset(Dataset):
    ''' Showing Telling Corpus Dataset '''
    def __init__(self, df): #데이터 전처리
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = int(self.df.iloc[idx, 0])
        return text, label

In [27]:
ST_train_dataset = STDataset(train_df)
train_loader = DataLoader(ST_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [28]:
ST_train_dataset.__getitem__(7)

('way door decisively slammed behind frightened instead looked upon spans green rolling hills bedecked colorful poppies tiny thatched roof cottages debated rolling plush hill feet lifting straight ground flying tiny village',
 1)

In [29]:
import torch

#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [30]:
for text, label in train_loader :
    print("text:",text)
    print("label:",label)

text: ('saw hammer dangled belt adze hooked one thumbnail black bowed saw several long wood shavings caught curly hair', 'mother switched light left room michael tensed huddled covers gripped sheets held breath wind brushed past curtain')
label: tensor([1, 1])
text: ('awaken annoying buzz alarm clock anything ready face day ahead drag', 'kicked open screen door letting slam wall dashed outside steps yard flew grabbing first rock path hurled back toward house crashed living room window explosion shattered glass')
label: tensor([1, 1])
text: ('cold carved bone deep fueled lash wind iced drowning rain gushing bloated sky iona welcome ireland loved could asked hugged arms chest drank wild soggy view window standing castle sleep castle night honest god castle heart west', 'leaves crunched beneath feet')
label: tensor([1, 1])
text: ('great conversation tim dinner loved hearing stories', 'took first bite hot slice piled high melted provolone thick pepperoni knew found small gslice h heaven')


In [32]:


optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1


/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [33]:
# evaluation
model.eval()

nsmc_eval_dataset = STDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)
 

/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test accuracy:  0.5
